In [1]:
import sys
sys.path.append("../")

import tensorflow as tf
from IJCAI19.test.AttackTest import *

InitGlobal()
TARGET_ATTACK = False


Using TensorFlow backend.


In [3]:
batch_shape = [FLAGS.batch_size, FLAGS.image_height, FLAGS.image_width, 3]
# Attack model
name = 'keras_xception_19'
attack_params = {"ep_ratio": 0.1, "nb_iter": 10, "target":TARGET_ATTACK}
T11 = AttackModel(batch_shape, FLAGS.num_classes, name=name)
A = EmbeddedAttackModel(batch_shape, FLAGS.num_classes)
A.add_model(T11)
Attack(A, GradSmoothMomentumIterativeMethod, attack_params, TARGET_ATTACK)

# Validate model
name = 'vgg_16'
T2 = AttackModel(batch_shape, FLAGS.num_classes, name=name)
Yp, topK, acc = Predict(T2, FLAGS.output_dir, targetlabel=TARGET_ATTACK)
X, Y, Xadv, Yadv = Score(Yp, TARGET_ATTACK)

loaded keras model from  ../IJCAI19/weight/xception_19/keras_xception_19.h5
{'ep_ratio': 0.1, 'nb_iter': 10, 'target': False, 'eps': 25.5, 'eps_iter': 2.55, 'y': <tf.Tensor 'output:0' shape=(?, 110) dtype=float32>, 'clip_min': 0, 'clip_max': 255}


..\IJCAI19\module\gs_mim.py:204: UserWarning: kwargs is unused and will be removed on or after 2019-04-26.
  warnings.warn("kwargs is unused and will be removed on or after "


AttributeError: Layer xception has multiple inbound nodes, hence the notion of "layer output" is ill-defined. Use `get_output_at(node_index)` instead.

In [10]:
img_loader = ImageLoader(FLAGS.input_dir, batch_shape, targetlabel=False, label_size=FLAGS.num_classes, format='png', label_file='dev.csv')

In [12]:
T11.model.cleverhans_model.get_logits(X)

AttributeError: Layer xception has multiple inbound nodes, hence the notion of "layer output" is ill-defined. Use `get_output_at(node_index)` instead.

In [11]:
_,X,Y = next(img_loader)

In [51]:
def finetune_xception_19(weights=None, input_shape=None, num_classes=None):
    base_weright = None
    fine_weight = None
    if weights:
        if weights == 'imagenet':
            base_weright = weights
        else:
            fine_weight = weights
    xception = Xception(weights=base_weright, include_top=False, input_shape=input_shape)
    finetune_layer_after = "add_11"
    trainable = False
    for layer in xception.layers:
        if layer.name == finetune_layer_after:
            trainable = True
        layer.trainable = trainable

    _input = xception.input
    x = xception.output
    x = layers.GlobalAveragePooling2D(name='final_gap')(x)
    _output = layers.Dense(num_classes,activation='softmax',kernel_initializer='he_normal', name='final_dense_mapping')(x)

    model = Model(inputs=_input, outputs=_output)
    if fine_weight:
        model.load_weights(fine_weight)
    return model

In [73]:
model = finetune_xception_19(input_shape=(299,299,3), num_classes=110)

In [53]:
model.load_weights('../IJCAI19/weight/xception_19/keras_xception_19.h5')

ValueError: You are trying to load a weight file containing 2 layers into a model with 81 layers.

In [62]:
old_layers = lm.layers[0].layers
for i in range(len(model.layers)):
    if model.layers[i].name == old_layers[i].name:
        model.layers[i].load_weights(old_layers[i].weights)
    else:
        print(i, model.layers[i].name, old_layers[i].name)

0 input_5 input_1


AttributeError: 'Conv2D' object has no attribute 'load_weights'

In [57]:
old_layers = lm.layers[0].layers

In [61]:
old_layers[10].weights

[<tf.Variable 'block2_sepconv2_3/depthwise_kernel:0' shape=(3, 3, 128, 1) dtype=float32_ref>,
 <tf.Variable 'block2_sepconv2_3/pointwise_kernel:0' shape=(1, 1, 128, 128) dtype=float32_ref>]

In [74]:
[l.name for l in model.layers]

['input_6',
 'block1_conv1',
 'block1_conv1_bn',
 'block1_conv1_act',
 'block1_conv2',
 'block1_conv2_bn',
 'block1_conv2_act',
 'block2_sepconv1',
 'block2_sepconv1_bn',
 'block2_sepconv2_act',
 'block2_sepconv2',
 'block2_sepconv2_bn',
 'conv2d_21',
 'block2_pool',
 'batch_normalization_21',
 'add_61',
 'block3_sepconv1_act',
 'block3_sepconv1',
 'block3_sepconv1_bn',
 'block3_sepconv2_act',
 'block3_sepconv2',
 'block3_sepconv2_bn',
 'conv2d_22',
 'block3_pool',
 'batch_normalization_22',
 'add_62',
 'block4_sepconv1_act',
 'block4_sepconv1',
 'block4_sepconv1_bn',
 'block4_sepconv2_act',
 'block4_sepconv2',
 'block4_sepconv2_bn',
 'conv2d_23',
 'block4_pool',
 'batch_normalization_23',
 'add_63',
 'block5_sepconv1_act',
 'block5_sepconv1',
 'block5_sepconv1_bn',
 'block5_sepconv2_act',
 'block5_sepconv2',
 'block5_sepconv2_bn',
 'block5_sepconv3_act',
 'block5_sepconv3',
 'block5_sepconv3_bn',
 'add_64',
 'block6_sepconv1_act',
 'block6_sepconv1',
 'block6_sepconv1_bn',
 'block6_se

In [70]:
[l.name for l in lm.layers[0].layers]

['input_1',
 'block1_conv1',
 'block1_conv1_bn',
 'block1_conv1_act',
 'block1_conv2',
 'block1_conv2_bn',
 'block1_conv2_act',
 'block2_sepconv1',
 'block2_sepconv1_bn',
 'block2_sepconv2_act',
 'block2_sepconv2',
 'block2_sepconv2_bn',
 'conv2d_1',
 'block2_pool',
 'batch_normalization_1',
 'add_1',
 'block3_sepconv1_act',
 'block3_sepconv1',
 'block3_sepconv1_bn',
 'block3_sepconv2_act',
 'block3_sepconv2',
 'block3_sepconv2_bn',
 'conv2d_2',
 'block3_pool',
 'batch_normalization_2',
 'add_2',
 'block4_sepconv1_act',
 'block4_sepconv1',
 'block4_sepconv1_bn',
 'block4_sepconv2_act',
 'block4_sepconv2',
 'block4_sepconv2_bn',
 'conv2d_3',
 'block4_pool',
 'batch_normalization_3',
 'add_3',
 'block5_sepconv1_act',
 'block5_sepconv1',
 'block5_sepconv1_bn',
 'block5_sepconv2_act',
 'block5_sepconv2',
 'block5_sepconv2_bn',
 'block5_sepconv3_act',
 'block5_sepconv3',
 'block5_sepconv3_bn',
 'add_4',
 'block6_sepconv1_act',
 'block6_sepconv1',
 'block6_sepconv1_bn',
 'block6_sepconv2_act

In [76]:
lm.layers[0].summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 149, 149, 32) 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 149, 149, 32) 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con